In [1]:
import random
import time
import pyautogui
import numpy as np
import cv2
from PIL import ImageGrab
import pytesseract
import win32gui
import win32con  
import keyboard
import threading
import pickle
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
TILE_COLORS = {
    (189, 172, 151): 0,     
    (238, 228, 218): 2,      
    (235, 216, 182): 4,      
    (242, 173, 114): 8,      
    (245, 142, 90): 16,      
    (246, 124, 95): 32,      
    (246, 94, 59): 64,      
    (243, 207, 82): 128,    
    (244, 204, 72): 256,    
    (237, 200, 80): 512,    
    (237, 197, 63): 1024,   
    (237, 194, 46): 2048,    
}

POSSIBLE_TITLES = ['2048 by Gabriele Cirulli • Play the Free Online Game']

def list_window_names():
    def callback(hwnd, names):
        if win32gui.IsWindowVisible(hwnd):
            names.append(win32gui.GetWindowText(hwnd))
    names = []
    win32gui.EnumWindows(callback, names)
    return [name for name in names if name]  

def get_window_rect(possible_titles):
    def callback(hwnd, result):
        if win32gui.IsWindowVisible(hwnd):
            title = win32gui.GetWindowText(hwnd).lower()
            for possible_title in possible_titles:
                if possible_title.lower() in title:
                    result.append((title, win32gui.GetWindowRect(hwnd)))
    result = []
    win32gui.EnumWindows(callback, result)
    if result:
        #print(f'Found window: {result[0][0]}')
        return result[0][1]
    return None

def activate_game_window():
    def callback(hwnd, hwnd_list):
        if win32gui.IsWindowVisible(hwnd):
            title = win32gui.GetWindowText(hwnd)
            if any(t.lower() in title.lower() for t in POSSIBLE_TITLES):
                hwnd_list.append(hwnd)
    
    hwnd_list = []
    win32gui.EnumWindows(callback, hwnd_list)
    if hwnd_list:
        game_hwnd = hwnd_list[0]
        win32gui.ShowWindow(game_hwnd, win32con.SW_RESTORE)
        win32gui.SetForegroundWindow(game_hwnd)
        return True
    return False

def cap_screen():
    if not activate_game_window():
        raise Exception('Could not activate 2048 game window')
    
    time.sleep(0.1)
    
    rect = get_window_rect(POSSIBLE_TITLES)
    if not rect:
        windows = list_window_names()
        print('Could not find 2048 window. Available windows:')
        print('\n'.join(f'- {w}' for w in windows))
        raise Exception('2048 game window not found. Please make sure the game is open.')

    x1, y1, x2, y2 = rect
    width = x2 - x1
    height = y2 - y1

    margin_x = width // 3     
    top_margin = height // 6  
    bottom_margin = height // 3 
    
    game_box = (x1 + margin_x,
               y1 + top_margin,
               x2 - margin_x,
               y2 - bottom_margin)

    button_box = (x1 + margin_x,
                 y2 - bottom_margin,
                 x2 - margin_x,
                 y2 - bottom_margin + bottom_margin//3)

    full_screen = np.array(ImageGrab.grab((x1, y1, x2, y2)))
    debug_screen = full_screen.copy()
    
    cv2.rectangle(debug_screen, 
                 (margin_x, top_margin),
                 (width - margin_x, height - bottom_margin),
                 (0, 255, 0), 2)
    cv2.rectangle(debug_screen,
                 (margin_x, height - bottom_margin),
                 (width - margin_x, height - bottom_margin + bottom_margin//3),
                 (255, 0, 0), 2)
    
    cv2.imshow('Full Window', cv2.cvtColor(debug_screen, cv2.COLOR_RGB2BGR))
    cv2.waitKey(1)

    game_screen = ImageGrab.grab(game_box)
    button_screen = ImageGrab.grab(button_box)
    
    return np.array(game_screen), np.array(button_screen)

In [3]:
WASD_KEYS = ['w', 'a', 's', 'd']

def press_key(key):
    pyautogui.keyDown(key)
    time.sleep(0.05)
    pyautogui.keyUp(key)
    time.sleep(0.15)

In [4]:
def encode_state(grid):
    return tuple(map(tuple, grid))

def get_reward(grid, prev_grid, game_over):
    if game_over:
        return -1000
    
    if 2048 in grid:
        return 5000
    
    current_sum = np.sum(grid)
    prev_sum = np.sum(prev_grid) if prev_grid is not None else 0
    merge_reward = current_sum - prev_sum
    
    curr_max = np.max(grid)
    prev_max = np.max(prev_grid) if prev_grid is not None else 0
    
    if curr_max > prev_max:
        max_tile_reward = curr_max * 2  
    else:
        max_tile_reward = curr_max * 0.1
    
    if np.array_equal(grid, prev_grid):
        return -10
    
    return merge_reward + max_tile_reward

def choose_action(state, q_table, epsilon):
    if random.random() < epsilon:
        return random.choice(WASD_KEYS)
    
    state_key = encode_state(state)
    if state_key not in q_table:
        q_table[state_key] = {move: 0 for move in WASD_KEYS}
    
    return max(q_table[state_key].items(), key=lambda x: x[1])[0]

def update_q_value(q_table, state, action, next_state, reward, learning_rate, discount_factor):
    state_key = encode_state(state)
    next_state_key = encode_state(next_state)
    
    if state_key not in q_table:
        q_table[state_key] = {move: 0 for move in WASD_KEYS}
    if next_state_key not in q_table:
        q_table[next_state_key] = {move: 0 for move in WASD_KEYS}
    
    old_value = q_table[state_key][action]
    next_max = max(q_table[next_state_key].values())
    new_value = (1 - learning_rate) * old_value + learning_rate * (reward + discount_factor * next_max)
    q_table[state_key][action] = new_value

def detect_game_over(button_image):
    gray = cv2.cvtColor(button_image, cv2.COLOR_RGB2GRAY)

    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY, 11, 2)

    configs = [
        '--psm 7',  
        '--psm 6', 
        '--psm 3'   
    ]

    for config in configs:
        text = pytesseract.image_to_string(thresh, config=config).strip().lower()
        if any(phrase in text for phrase in ['game', 'over', 'try', 'again', 'play']):
            debug_image = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
            cv2.putText(debug_image, f"Detected: {text}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            cv2.imshow('Game Over Detection', debug_image)
            cv2.waitKey(1)
            return True

        inverted = cv2.bitwise_not(thresh)
        text = pytesseract.image_to_string(inverted, config=config).strip().lower()
        if any(phrase in text for phrase in ['game', 'over', 'try', 'again', 'play']):
            return True

    return False

def click_play_again():
    rect = get_window_rect(POSSIBLE_TITLES)
    if not rect:
        raise Exception('2048 game window not found')
        
    x1, y1, x2, y2 = rect
    width = x2 - x1
    height = y2 - y1
    
    bottom_margin = height // 3
    
    button_center_x = x1 + (width // 2)
    button_center_y = y2 - bottom_margin + (bottom_margin // 6)
    
    pyautogui.moveTo(button_center_x, button_center_y)
    time.sleep(1)
    pyautogui.click()
    time.sleep(0.3) 

In [5]:
def parse_grid(screen):
    height, width = screen.shape[:2]
    cell_width = width // 4
    cell_height = height // 4
    grid = np.zeros((4, 4), dtype=int)
    debug_screen = screen.copy()
    
    MAX_COLOR_DISTANCE = 100
    
    def get_tile_color(x1, y1, x2, y2):
        region = screen[y1:y2, x1:x2]
        pixels = region.reshape(-1, 3)
        num_samples = len(pixels) // 2
        sample_indices = np.random.choice(len(pixels), num_samples, replace=False)
        samples = pixels[sample_indices]
        median_color = tuple(np.median(samples, axis=0).astype(int))
        return median_color
    
    for i in range(4):
        for j in range(4):
            x1 = j * cell_width
            y1 = i * cell_height
            x2 = (j + 1) * cell_width
            y2 = (i + 1) * cell_height
            
            color = get_tile_color(x1, y1, x2, y2)
            
            min_dist = float('inf')
            best_value = 0
            
            for tile_color, value in TILE_COLORS.items():
                dist = sum((a - b) ** 2 for a, b in zip(color, tile_color))
                if dist < min_dist:
                    min_dist = dist
                    best_value = value
            
            grid[i, j] = best_value if min_dist <= MAX_COLOR_DISTANCE else 0
            
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            cv2.putText(debug_screen,
                      f"{grid[i,j]}\n{color}",
                      (center_x - 30, center_y),
                      cv2.FONT_HERSHEY_SIMPLEX,
                      0.4, (0, 0, 255), 1)
            cv2.rectangle(debug_screen,
                       (x1, y1), (x2, y2),
                       (0, 255, 0), 1)
    
    cv2.imshow('Grid Detection Debug', cv2.cvtColor(debug_screen, cv2.COLOR_RGB2BGR))
    cv2.waitKey(1)
    
    return grid

### MAIN CODE

In [12]:
def save_q_table(q_table, filename='q_table.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(q_table, f)

def load_q_table(filename='q_table.pkl'):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
    return {}

print('Please open the 2048 game in your browser')
print('You have 5 seconds to switch to the game window...')
print('Press k to stop the program at any time')
time.sleep(5)

stop_flag = False

def check_keyboard():
    global stop_flag
    while not stop_flag:
        if keyboard.is_pressed('k'):
            print('\nStopping program...')
            stop_flag = True
        time.sleep(0.1)

keyboard_thread = threading.Thread(target=check_keyboard)
keyboard_thread.daemon = True
keyboard_thread.start()

q_table = load_q_table()
discount_factor = 0.9
epsilon=0.85
learning_rate=0.7
games_played = 0

try:
    while not stop_flag:
        screen, button_area = cap_screen()
        grid = parse_grid(screen)
        print('Initial grid state:')
        print(grid)
        
        if games_played == 0:
            input('Is grid detection correct? Press Enter to start playing, Ctrl+C to abort...')
            time.sleep(5)
        
        last_grid = None
        game_running = True
        repeated_moves = 0
        last_action = None
        
        while game_running and not stop_flag:
            screen, button_area = cap_screen()
            time.sleep(0.1)
            grid = parse_grid(screen)

           
            if last_action and action == last_action:
                repeated_moves += 1
                if repeated_moves >= 3:
                    print('Forcing different move after 3 repeated attempts')
                    available_moves = [m for m in WASD_KEYS if m != action]
                    action = random.choice(available_moves)
                    repeated_moves = 0
                    continue
            else:
                repeated_moves = 0

            if 2048 in grid:
                print('2048 achieved! Congratulations!')
                stop_flag = True
                break
            
            if detect_game_over(button_area):
                print('Game Over - Play Again detected!')
                games_played += 1
                learning_rate+=0.05
                if(epsilon>0.2):
                    epsilon-=0.05
                click_play_again()
                time.sleep(1)
                continue
            
            state = encode_state(grid)
            action = choose_action(grid, q_table, epsilon)
            print(f'Action: {action} | Grid:')
            print(grid)
            last_action = action
            press_key(action)
            time.sleep(0.2)
            screen, _ = cap_screen()
            time.sleep(0.1)
            next_grid = parse_grid(screen)
            
            reward = get_reward(next_grid, grid, detect_game_over(button_area))
            update_q_value(q_table, grid, action, next_grid, reward, learning_rate, discount_factor)
            
            last_grid = grid.copy()
            time.sleep(0.1)
        
        save_q_table(q_table)
        print(f'Games played: {games_played}')
        
except Exception as e:
    print(f'Error: {e}')
    print('Please make sure the 2048 game is open in your browser')
finally:
    stop_flag = True
    keyboard_thread.join(timeout=1.0)
    save_q_table(q_table)
    print(f'Learning saved. Total games played: {games_played}')

Please open the 2048 game in your browser
You have 5 seconds to switch to the game window...
Press k to stop the program at any time
Initial grid state:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Action: d | Grid:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 2 0]
 [0 0 2 0]]
Action: w | Grid:
[[0 0 0 0]
 [0 0 0 0]
 [2 0 0 2]
 [0 0 0 2]]
Action: w | Grid:
[[2 0 0 4]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 2]]
Forcing different move after 3 repeated attempts
Action: a | Grid:
[[2 0 2 4]
 [0 0 0 2]
 [0 0 0 0]
 [0 0 0 0]]
Action: d | Grid:
[[4 4 0 0]
 [2 2 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Action: a | Grid:
[[0 0 0 8]
 [0 0 0 4]
 [0 0 0 2]
 [0 0 0 0]]
Forcing different move after 3 repeated attempts
Action: a | Grid:
[[8 0 0 0]
 [4 0 2 0]
 [2 0 0 0]
 [0 0 0 0]]
Action: d | Grid:
[[8 0 0 2]
 [4 2 0 0]
 [2 0 0 0]
 [0 0 0 0]]
Action: d | Grid:
[[0 0 8 2]
 [0 0 4 2]
 [0 2 0 2]
 [0 0 0 0]]
Forcing different move after 3 repeated attempts
Action: s | Grid:
[[0 0 8 2]
 [0 0 4 2]
 [0 0 0 4]
 [0 4 0 0]]
Action: s | Grid:
[[0 0 0

### TEST AREA

In [ ]:

time.sleep(5)

if detect_game_over(button_area):

    click_play_again()



In [ ]:
time.sleep(5)

rect = get_window_rect(POSSIBLE_TITLES)
if not rect:
    raise Exception('2048 game window not found')
        
x1, y1, x2, y2 = rect
width = x2 - x1
height = y2 - y1
    
bottom_margin = height // 3
    
button_center_x = x1 + (width // 2)
button_center_y = y2 - bottom_margin + (bottom_margin // 6)
    
pyautogui.moveTo(button_center_x, button_center_y)
time.sleep(1)
pyautogui.click()
time.sleep(0.3) 

Found window: 2048 by gabriele cirulli • play the free online game — mozilla firefox


In [ ]:


try:
    epsilon = 0.9
    print(f"Initial epsilon: {epsilon}")  
    if epsilon > 0.1:
        epsilon -= 0.05
        print(f"Updated epsilon: {epsilon}")  
except Exception as e:
    print(f"An error occurred: {e}")  

Initial epsilon: 0.9
Updated epsilon: 0.85
